In [34]:
import pandas as pd

In [35]:
df = pd.read_parquet('../cache/merged.parquet')

In [36]:
df.head()

,Date,Timestamp,Actor1Country,Actor1GeoCountry,Actor1Type,Actor2Country,Actor2GeoCountry,Actor2Type,ActionCountry,EventType,GoldsteinScale,NumSources,NumArticles,AvgTone,Magnitude,Impact,Impact_bin,pct_change_15min,pct_change_30min,pct_change_24h
0,2019-01-01,1546300800,None,None,CVL,None,None,None,None,Diplom Coop,3.4,4,50,2.354384,6.85,2.33,Positive,-0.033061,-0.226363,-2.433464
1,2019-01-01,1546300800,BRA,BR,None,None,None,None,BR,Intent to Coop,4.0,3,30,-0.339542,3.34,1.34,Slightly Positive,-0.033061,-0.226363,-2.433464
2,2019-01-01,1546300800,None,US,GOV,None,US,GOV,US,Appeal,3.2,10,40,1.734061,8.73,2.79,Positive,-0.033061,-0.226363,-2.433464
3,2019-01-01,1546300800,USA,US,None,None,None,None,US,Disapprove,-2.0,3,21,-4.170072,6.72,-1.34,Slightly Negative,-0.033061,-0.226363,-2.433464
4,2019-01-01,1546300800,USA,US,EDU,None,None,None,US,Consult,1.9,6,18,1.415701,5.32,1.01,Slightly Positive,-0.033061,-0.226363,-2.433464


In [ ]:
quants = [0, 0.5, 0.9, 0.99]
mag_quantiles = [df['Magnitude'].quantile(quant) for quant in quants]

df['AbsChange'] = df['pct_change_30min'].abs()
avg_change = df.AbsChange.mean()
catg_features = ['Actor1Country', 'Actor1GeoCountry', 'Actor1Type',
                 'Actor2Country', 'Actor2GeoCountry', 'Actor2Type',
                 'ActionCountry', 'EventType']

num_to_keep = 8
dfs = [pd.DataFrame(df) for _ in range(len(catg_features))]

for feature in catg_features:
    for i, mag in enumerate(mag_quantiles):
        df_sub_orig = df[df.Magnitude > mag].astype(object)
        top_countries = df_sub_orig[feature].value_counts().index[:40]
        df_sub = df_sub_orig.groupby(feature).filter(lambda x: x.name in top_countries)
        df_grouped = (df_sub.groupby([feature], observed=True).AbsChange.mean() - avg_change).abs().sort_values(ascending=False)
        df_grouped = df_grouped[:num_to_keep]
        new_df = dfs[i]
        new_df[feature] = new_df[feature].fillna('Missing')
        new_df[feature] = new_df[feature].apply(lambda x: x if x in df_grouped.index else 'Other')
        dfs[i] = new_df
    print(f'Finished {feature}')

In [15]:
merged_0q = dfs[0] # 0 quantile (all data)
merged_50q = dfs[1] # 0.5 quantile
merged_90q = dfs[2] # 0.9 quantile
merged_99q = dfs[3] # 0.99 quantile

In [32]:
merged_0q.EventType.cat = merged_0q.EventType.cat.categories.append(pd.Index(['Missing', 'Other']))

In [33]:
merged_0q['EventType'].value_counts()

EventType
Consult              1267084
Statement             648289
Diplom Coop           419667
Appeal                418795
Intent to Coop        368722
Fight                 345688
Disapprove            305588
Coerce                255777
Yield                 202861
Aid                   174983
Reject                146931
Investigate           136693
Material Coop         106830
Assault                81196
Threaten               80823
Demand                 69028
Reduce Relations       60881
Protest                59635
Demonstrate Force      20461
Mass Violence           1609
Name: count, dtype: int64

In [ ]:
merged_50q.head()

In [ ]:
merged_90q.head()

In [ ]:
merged_99q.head()

In [ ]:
merged_0q.to_parquet('../cache/merged_0q.parquet')
merged_50q.to_parquet('../cache/merged_50q.parquet')
merged_90q.to_parquet('../cache/merged_90q.parquet')
merged_99q.to_parquet('../cache/merged_99q.parquet')